In [2]:
from pymilvus import MilvusClient

host = "localhost"
port = "19530"

milvus_client = MilvusClient(host=host, port=port)

In [3]:
from pymilvus import FieldSchema, DataType, CollectionSchema

VECTOR_LENGTH = 768  # check the dimensionality for Silver Retriever Base (v1.1) model

id_field = FieldSchema(
    name="id", dtype=DataType.INT64, is_primary=True, description="Primary id"
)
text = FieldSchema(
    name="text", dtype=DataType.VARCHAR, max_length=4096, description="Page text"
)
embedding_text = FieldSchema(
    "embedding",
    dtype=DataType.FLOAT_VECTOR,
    dim=VECTOR_LENGTH,
    description="Embedded text",
)

fields = [id_field, text, embedding_text]

schema = CollectionSchema(
    fields=fields,
    auto_id=True,
    enable_dynamic_field=True,
    description="RAG Texts collection",
)

In [ ]:
COLLECTION_NAME = "rag_texts_and_embeddings"

milvus_client.create_collection(collection_name=COLLECTION_NAME, schema=schema)

index_params = milvus_client.prepare_index_params()

index_params.add_index(
    field_name="embedding",
    index_type="HNSW",
    metric_type="L2",
    params={"M": 4, "efConstruction": 64},  # lower values for speed
)

milvus_client.create_index(collection_name=COLLECTION_NAME, index_params=index_params)

# checkout our collection
print(milvus_client.list_collections())

# describe our collection
print(milvus_client.describe_collection(COLLECTION_NAME))

In [ ]:
# define data source and destination
## the document origin destination from which document will be downloaded
pdf_url = "https://www.iab.org.pl/wp-content/uploads/2024/04/Przewodnik-po-sztucznej-inteligencji-2024_IAB-Polska.pdf"

## local destination of the document
file_name = "Przewodnik-po-sztucznej-inteligencji-2024_IAB-Polska.pdf"

## local destination of the processed document
file_json = "Przewodnik-po-sztucznej-inteligencji-2024_IAB-Polska.json"

## local destination of the embedded pages of the document
embeddings_json = "Przewodnik-po-sztucznej-inteligencji-2024_IAB-Polska-Embeddings.json"

## local destination of all above local required files
data_dir = "./data"

In [ ]:
# download data
import os
import requests


def download_pdf_data(pdf_url: str, file_name: str) -> None:
    response = requests.get(pdf_url, stream=True)
    with open(os.path.join(data_dir, file_name), "wb") as file:
        for block in response.iter_content(chunk_size=1024):
            if block:
                file.write(block)


download_pdf_data(pdf_url, file_name)

In [ ]:
# prepare data

import fitz
import json


def extract_pdf_text(file_name, file_json):
    document = fitz.open(os.path.join(data_dir, file_name))
    pages = []

    for page_num in range(len(document)):
        page = document.load_page(page_num)
        page_text = page.get_text()
        pages.append({"page_num": page_num, "text": page_text})

    with open(os.path.join(data_dir, file_json), "w") as file:
        json.dump(pages, file, indent=4, ensure_ascii=False)


extract_pdf_text(file_name, file_json)

In [6]:
# vectorize data

import torch
from sentence_transformers import SentenceTransformer


def generate_embeddings(file_json, embeddings_json, model):
    pages = []
    with open(os.path.join(data_dir, file_json), "r") as file:
        data = json.load(file)

    for page in data:
        pages.append(page["text"])

    embeddings = model.encode(pages)

    embeddings_paginated = []
    for page_num in range(len(embeddings)):
        embeddings_paginated.append(
            {"page_num": page_num, "embedding": embeddings[page_num].tolist()}
        )

    with open(os.path.join(data_dir, embeddings_json), "w") as file:
        json.dump(embeddings_paginated, file, indent=4, ensure_ascii=False)


model_name = "ipipan/silver-retriever-base-v1.1"
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer(model_name, device=device)
generate_embeddings(file_json, embeddings_json, model)

In [ ]:
def insert_embeddings(file_json, embeddings_json, client=milvus_client):
    rows = []
    with (
        open(os.path.join(data_dir, file_json), "r") as t_f,
        open(os.path.join(data_dir, embeddings_json), "r") as e_f,
    ):
        text_data, embedding_data = json.load(t_f), json.load(e_f)
        text_data = list(map(lambda d: d["text"], text_data))
        embedding_data = list(map(lambda d: d["embedding"], embedding_data))

        for page, (text, embedding) in enumerate(zip(text_data, embedding_data)):
            rows.append({"text": text, "embedding": embedding})

    client.insert(collection_name="rag_texts_and_embeddings", data=rows)


insert_embeddings(file_json, embeddings_json)

# load inserted data into memory
milvus_client.load_collection("rag_texts_and_embeddings")

In [7]:
# search
def search(model, query, client=milvus_client):
    embedded_query = model.encode(query).tolist()
    result = client.search(
        collection_name="rag_texts_and_embeddings",
        data=[embedded_query],
        limit=1,
        search_params={"metric_type": "L2"},
        output_fields=["text"],
    )
    return result


result = search(model, query="Czym jest sztuczna inteligencja")

In [ ]:
from google import genai

GEMINI_KEY = os.getenv("GEMINI_API_KEY")
gemini_client = genai.Client(api_key=GEMINI_KEY)

MODEL = "gemini-2.0-flash"


def generate_response(prompt: str):
    try:
        # Send request to Gemini 2.0 Flash API and get the response
        response = gemini_client.models.generate_content(
            model=MODEL,
            contents=prompt,
        )
        return response.text
    except Exception as e:
        print(f"Error generating response: {e}")
        return None

In [13]:
def build_prompt(context: str, query: str) -> str:
    prompt = f"""
Jesteś inteligentnym asystentem, który odpowiada na pytania na podstawie dostarczonego kontekstu.
Użyj wyłącznie informacji z kontekstu. Jeśli kontekst nie zawiera wystarczających danych, napisz, że brakuje informacji.

Kontekst:
{context}

Pytanie:
{query}

Odpowiedź:
"""
    return prompt


def rag(model, query: str) -> str:
    # having all prepared functions, you can combine them together and try to build your own RAG!
    search_result = search(model, query=query)

    if not search_result or not search_result[0]:
        return "No relevant context found."

    retrieved_texts = [hit["entity"]["text"] for hit in search_result[0]]
    context = "\n\n".join(retrieved_texts)

    prompt = build_prompt(context=context, query=query)

    answer = generate_response(prompt)

    return answer or "No response generated."

In [14]:
query = "Czym jest sztuczna inteligencja?"
response = rag(model, query)
print(response)

Sztuczna inteligencja (AI) to obszar informatyki, który skupia się na tworzeniu programów komputerowych zdolnych do wykonywania zadań, które wymagają ludzkiej inteligencji. Te zadania obejmują rozpoznawanie wzorców, rozumienie języka naturalnego, podejmowanie decyzji, uczenie się, planowanie i wiele innych. Głównym celem AI jest stworzenie systemów, które są zdolne do myślenia i podejmowania decyzji na sposób przypominający ludzki.

